# GeoTagger - Spacy

In [1]:
# Import necessary libraries.
import re, warnings, urllib, requests, spacy, geopy, folium, os, sys, glob
import pandas as pd
import numpy as np
from collections import Counter
from geopy.extra.rate_limiter import RateLimiter

# Import project-specific functions. 
# Python files (.py) have to be in same folder to work.
lib_path = os.path.abspath(os.path.join(os.path.dirname('Correspondence_XML_parser.py'), '../Scripts'))
sys.path.append(lib_path)

from Correspondence_XML_parser import *

nlp = spacy.load('en_core_web_sm')

# Ignore warnings related to deprecated functions.
warnings.filterwarnings('ignore')

## Get XML Files

In [2]:
%%time

# Declare directory location to shorten filepaths later.
abs_dir = "/Users/quinn.wi/Documents/Data"
files = glob.glob(abs_dir + "/PSC/Sedgwick/*.xml")

len(files)

CPU times: user 1.06 ms, sys: 1.32 ms, total: 2.38 ms
Wall time: 2.78 ms


122

## Build Dataframe

In [3]:
%%time

# Build dataframe from XML files.
# build_dataframe() called from Correspondence_XML_parser
df = build_dataframe(files)

df.head(3)

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-04-26-toFrancesSedgwickWatsonFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1803-10-06-toPamelaDwightSedgwickF.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1809-01-27-toTheodoreSedgwickIFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-12-25-toFrancesSedgwickWatsonFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1806-01-17-toPamelaDwightSedgwickFD (1).xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1805-11-29-toPamelaDwightSedgwickFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-04-26-toFSWF.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1800-01-12-toTheodoreSedgwickIF.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1805-11-15-toPamelaDwightSedgwickFD (1).xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-12-28-toFrancesSedgwickWatsonFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-03-24-toFrancesSedgwickWatsonFD.xml 

/Users/quinn.wi/Documents/Data/PSC/

,file,date,source,target,subjects,references,text
0,CMS1819-03-08-toRobertSedgwickIF (1).xml,1819-03-08,Catharine Maria Sedgwick,sedgwick-robert,,"sedgwick-charles,sedgwick-elizabeth,sedgwick-h...",Albany March 8' 1819 -- I came here my dear Ro...
1,CMS1816-03-25-toFrancesSedgwickWatsonF.xml,1816-03-25,Catharine Maria Sedgwick,FSW,,"RSI,banyer-maria,jay-sarah,van vechten-jacob,s...",Albany March 25th 1816 I have just heard of an...
2,CMS1813-08-15-toRobertSedgwickIF.xml,1813-08-15,Catharine Maria Sedgwick,RSI,,"FSW,U,payne-eloise,warner-thomas,warner-france...",Stockbridge August 15th 1813 I recollect very...


## Get Place Names

In [ ]:
%%time

def get_placenames(text):
    doc = nlp(text)
    places = [ent.text for ent in doc.ents if ent.label_ in ['LOC', 'GPE']]
    return places
    
df['places'] = df['text'].apply(lambda x: get_placenames(x))

df = df[['file', 'date', 'places']]
df = df.explode('places')

df.head(3)

## GeoCode Places

In [5]:
%%time

# https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/
def geocode(place):
#     url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(place) +'?format=json'
    url = 'https://nominatim.openstreetmap.org/search/' + str(place) + '?format=json'
    response = requests.get(url).json()
    if (len(response) != 0):
#         Default (response[0]): select first search hit in OpenStreetMap.
        return (float(response[0]['lat']), float(response[0]['lon']))
    else:
        return None 

df['coordinates'] = df['places'].apply(geocode)
df[['lat', 'lon']] = pd.DataFrame(df['coordinates'].tolist(), index = df.index)

# Convert to floats.
df['lat'] = df['lat'].apply(lambda x: float(x))
df['lon'] = df['lon'].apply(lambda x: float(x))

df = df.dropna()

df.head(3)

CPU times: user 1.23 s, sys: 101 ms, total: 1.34 s
Wall time: 1min 7s


,file,date,source,target,subjects,references,text,places,coordinates,lat,lon
0,CMS1819-03-08-toRobertSedgwickIF (1).xml,1819-03-08,Catharine Maria Sedgwick,sedgwick-robert,,"sedgwick-charles,sedgwick-elizabeth,sedgwick-h...",Albany March 8' 1819 -- I came here my dear Ro...,"[Albany, Northampton, Boston]","(42.3402826, -71.0828185)",42.340283,-71.082819
1,CMS1816-03-25-toFrancesSedgwickWatsonF.xml,1816-03-25,Catharine Maria Sedgwick,FSW,,"RSI,banyer-maria,jay-sarah,van vechten-jacob,s...",Albany March 25th 1816 I have just heard of an...,[Providence],"(41.8239891, -71.4128343)",41.823989,-71.412834
4,CMS1818-11-04-toRobertSedgwickIF.xml,1818-11-04,Catharine Maria Sedgwick,sedgwick-robert,,"sedgwick-susan,pomeroy-eliza,sedgwick-jane,sed...",Stockbridge Nov 4 Wed Tuesday Morn'g --My dear...,"[Morocco, Albany]","(21.9128448, -102.2782706)",21.912845,-102.278271


### Save Results

In [7]:
%%time

df.to_csv(os.path.abspath('../../lab_space/projects/sedgwick/geo/data/sedgwick_geoReference.csv'), 
                          sep = ',', index = False)

CPU times: user 6.33 ms, sys: 2 ms, total: 8.33 ms
Wall time: 7.71 ms
